In [1]:
import sys
import os

current_dir = os.path.abspath(os.getcwd())
project_home_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

sys.path.append(project_home_dir)

In [2]:
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import wandb

from src.data.dataset_loader import load_custom_dataset, splited_dataset
from src.model.training import train_model

In [3]:
root_dir = "data/raw/_4_classi"

wandb.init(project="CowSegmentation", name="SegFormer")

dataset = load_custom_dataset(root_dir)


In [4]:
from torch.utils.data import DataLoader

dataset_splited = splited_dataset(dataset)

train_loader = DataLoader(dataset_splited['train'], batch_size=16, shuffle=True)
val_loader = DataLoader(dataset_splited['validation'], batch_size=16, shuffle=False)

In [5]:
model_name = "nvidia/mit-b0"
num_epochs = 100
num_classes = 4  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(
    model_name,
    num_labels=num_classes,
    ignore_mismatched_sizes=True  
).to(device)

In [7]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

In [9]:
train_model(model=model, train_loader=train_loader, val_loader=val_loader, criterion=criterion, optimizer=optimizer, num_epochs=num_epochs, device=device)